# Code to build yield curve


In [1]:
# Reading the excel and setting up data
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.optimize import minimize, brentq
import plotly.graph_objects as go
import plotly.io as pio
import io

pio.templates.default = "plotly_white"


df_raw = pd.read_excel("data.xlsx")


### Data preprocessing

In [ ]:
# Set the date for our analysis this is around the last coupon payment
VALUATION_DATE = datetime(2025, 2, 28)

# setting our actual prices and extracting maturity
df = df_raw.copy()
df['Issue Date'] = pd.to_datetime(df['Issue Date'])
df['Maturity'] = pd.to_datetime(df['Maturity'])
for col in ['Bid Price', 'Ask Price', 'Cpn']:
    df[col] = pd.to_numeric(df[col])

# we use mid price as market price
df['Market_Price'] = (df['Bid Price'] + df['Ask Price']) / 2

# We find maturity in years using date time 
# We use 365.25 to account for leap years on average.
df['Maturity_in_Years'] = (df['Maturity'] - VALUATION_DATE).dt.days / 365.25


def generate_cashflows(bond, valuation_date):
    """Generates a list of future cash flows for a single bond"""
    
    cashflows = []
    coupon_amount = bond['Cpn'] / 2.0
    maturity_date = bond['Maturity']
    
    # final payment is 100 + coupon
    cashflows.append((maturity_date, 100.0 + coupon_amount))
    
    # Start from the maturity date and step backwards in 6-month intervals
    # to find all coupon payment dates before the valuation date, we use relative delta 
    # to use date time to our advantae
    
    
    payment_date = maturity_date
    while True:
        payment_date -= relativedelta(months=6)
        if payment_date <= valuation_date:
            break
        cashflows.append((payment_date, coupon_amount))
        
    # Convert payment dates to time in years from the valuation date
    cashflows_in_years = []
    for p_date, amount in cashflows:
        time_to_payment = (p_date - valuation_date).days / 365.25
        cashflows_in_years.append((time_to_payment, amount))
        
    # Return sorted by time, which is crucial for pricing
    return sorted(cashflows_in_years, key=lambda x: x[0])

# Apply this function to every bond (row) in our DataFrame.
df['Cashflows'] = df.apply(generate_cashflows, axis=1, valuation_date=VALUATION_DATE)

print(f"Data pre-processing complete. Valuation Date: {VALUATION_DATE.strftime('%Y-%m-%d')}")
print("\nDataFrame with generated cash flows:")
df[['Maturity', 'Market_Price', 'Maturity_in_Years', 'Cashflows']].head()
df

Data pre-processing complete. Valuation Date: 2025-02-28

DataFrame with generated cash flows:


,Issuer Name,Issue Date,Maturity,Bid Price,Ask Price,Cpn,Currency,Market_Price,Maturity_in_Years,Cashflows
0,United States Treasury Note/Bond,2018-02-28,2025-02-28,99.980469,100.007812,2.750,USD,99.994141,0.000000,"[(0.0, 101.375)]"
1,United States Treasury Note/Bond,2023-02-28,2025-02-28,99.984375,100.019531,4.625,USD,100.001953,0.000000,"[(0.0, 102.3125)]"
2,United States Treasury Note/Bond,2020-03-02,2025-02-28,99.945312,100.011719,1.125,USD,99.978516,0.000000,"[(0.0, 100.5625)]"
3,United States Treasury Note/Bond,2022-03-15,2025-03-15,99.859375,99.914062,1.750,USD,99.886719,0.041068,"[(0.04106776180698152, 100.875)]"
4,United States Treasury Note/Bond,2020-03-31,2025-03-31,99.621094,99.703125,0.500,USD,99.662109,0.084873,"[(0.08487337440109514, 100.25)]"
...,...,...,...,...,...,...,...,...,...,...
342,United States Treasury Note/Bond,2024-02-15,2054-02-15,93.375000,93.421875,4.250,USD,93.398438,28.963723,"[(0.459958932238193, 2.125), (0.96372347707049..."
343,United States Treasury Note/Bond,2024-05-15,2054-05-15,99.437500,99.484375,4.625,USD,99.460938,29.207392,"[(0.2080766598220397, 2.3125), (0.711841204654..."
344,United States Treasury Note/Bond,2024-08-15,2054-08-15,93.531250,93.578125,4.250,USD,93.554688,29.459274,"[(0.459958932238193, 2.125), (0.96372347707049..."
345,United States Treasury Note/Bond,2024-11-15,2054-11-15,97.625000,97.656250,4.500,USD,97.640625,29.711157,"[(0.2080766598220397, 2.25), (0.71184120465434..."


### Step 3: Defining the Nelson-Siegel Model

The Nelson-Siegel model describes the entire yield curve with just four parameters. This makes it powerful and efficient.

The formula for the continuously compounded spot rate `r(t)` at time `t` is:

`r(t) = β₀ + β₁ * [(1 - e^(-t/τ)) / (t/τ)] + β₂ * [(1 - e^(-t/τ)) / (t/τ) - e^(-t/τ)]`

The four parameters have intuitive financial interpretations:
- **`β₀` (beta0):** The long-run interest rate. As maturity `t` goes to infinity, the curve approaches this level.
- **`β₁` (beta1):** The short-term slope. It determines the difference between the very short-term rate and the long-term rate.
- **`β₂` (beta2):** The curvature. It introduces a "hump" or "trough" in the middle of the curve.
- **`τ` (tau):** The scale parameter. It controls where the maximum curvature occurs.

We will implement two functions:
1. `nelson_siegel_rate()`: Directly implements the formula above.
2. `price_bond_with_ns()`: Calculates a bond's theoretical price by taking its cash flows and discounting each one back to the present using the appropriate spot rate from our Nelson-Siegel curve.

In [4]:
def nelson_siegel_rate(t, beta0, beta1, beta2, tau):
    """Calculates the spot rate for time 't' using the Nelson-Siegel formula."""
    # Handle the edge case of t=0 to avoid division by zero
    if t < 1e-6:
        return beta0 + beta1
        
    term1 = beta0
    term2 = beta1 * (1 - np.exp(-t / tau)) / (t / tau)
    term3 = beta2 * ((1 - np.exp(-t / tau)) / (t / tau) - np.exp(-t / tau))
    
    return term1 + term2 + term3

def price_bond_with_ns(cashflows, beta0, beta1, beta2, tau):
    """
    Calculates the theoretical price of a bond by discounting its cash flows
    using rates from the Nelson-Siegel curve.
    """
    total_present_value = 0.0
    for t, cf_amount in cashflows:
        # Get the spot rate for this specific cash flow's time-to-payment
        rate = nelson_siegel_rate(t, beta0, beta1, beta2, tau)
        # Discount the cash flow using continuous compounding: PV = C * e^(-r*t)
        total_present_value += cf_amount * np.exp(-rate * t)
        
    return total_present_value

### Step 4: Creating the Objective Function

Now we need a way to measure how well a given set of four parameters (`β₀, β₁, β₂, τ`) fits the market data. We do this with an **objective function** (or error function).

The function will:
1. Take a set of parameters as input.
2. Loop through every bond in our dataset.
3. For each bond, calculate its theoretical price using our `price_bond_with_ns` function.
4. Calculate the difference between this model price and the actual `Market_Price`.
5. Square this difference (to make it positive and to penalize larger errors more heavily).
6. Sum up all these squared differences.

The goal of the optimization process in the next step will be to find the parameters that make the output of this function as small as possible.

In [5]:
def objective_function(params, df_bonds):
    """
    The function to be minimized. It calculates the sum of squared differences
    between model prices and market prices for all bonds.
    """
    beta0, beta1, beta2, tau = params
    total_squared_error = 0.0
    
    for _, bond in df_bonds.iterrows():
        model_price = price_bond_with_ns(bond['Cashflows'], beta0, beta1, beta2, tau)
        market_price = bond['Market_Price']
        
        error = (model_price - market_price) ** 2
        total_squared_error += error
        
    return total_squared_error

# --- Test run of the function ---
# This confirms our functions are working before optimization.
initial_guess_params = [0.03, -0.01, 0.01, 1.5] # [beta0, beta1, beta2, tau]
initial_error = objective_function(initial_guess_params, df)

print(f"Objective function is ready.")
print(f"Test run: Initial error with guess parameters {initial_guess_params} is: {initial_error:.4f}")

Objective function is ready.
Test run: Initial error with guess parameters [0.03, -0.01, 0.01, 1.5] is: 52508.2089


### Step 5: Finding the Optimal Parameters

This is where we use `scipy.optimize.minimize` to do the heavy lifting. We provide it with:
- `fun`: Our `objective_function` to minimize.
- `x0`: An `initial_guess` for the parameters to start the search.
- `args`: Any extra arguments our function needs (in this case, the bond DataFrame).
- `method`: The optimization algorithm to use. 'L-BFGS-B' is a good choice as it's efficient and respects bounds.
- `bounds`: Constraints on the parameters. This is crucial for guiding the optimizer and preventing it from trying nonsensical values (e.g., a negative `τ` or an extreme interest rate).

In [ ]:
# Set bounds to guide the optimizer.
# (beta0, beta1, beta2, tau)
bounds = [
    (0, 0.2),      # beta0 (long-term rate) should be between 0% and 20%
    (-0.2, 0.2),   # beta1 (slope) can be negative or positive
    (-0.2, 0.2),   # beta2 (curvature) can be negative or positive
    (0.1, 30.0)    # tau (scale) must be positive and is typically in this range
]

print("Starting optimization to fit Nelson-Siegel parameters...")

# Run the optimizer
optimization_result = minimize(
    fun=objective_function,     # The function to minimize
    x0=initial_guess_params,    # The starting guess
    args=(df,),                 # Extra arguments to pass to our function
    method='L-BFGS-B',
    bounds=bounds  
)

# Extract and display the results
if optimization_result.success:
    optimal_params = optimization_result.x
    final_error = optimization_result.fun
    print("\nOptimization successful!")
    print(f"Final minimized error: {final_error:.4f}")
    print("\nOptimal Parameters:")
    print(f"  β₀ (beta0): {optimal_params[0]:.6f}")
    print(f"  β₁ (beta1): {optimal_params[1]:.6f}")
    print(f"  β₂ (beta2): {optimal_params[2]:.6f}")
    print(f"  τ  (tau):   {optimal_params[3]:.6f}")
else:
    print("\nOptimization failed to converge.")
    optimal_params = initial_guess_params # Fallback to guess if needed

Starting optimization to fit Nelson-Siegel parameters...

Optimization successful!
Final minimized error: 142.4884

Optimal Parameters:
  β₀ (beta0): 0.049755
  β₁ (beta1): 0.004015
  β₂ (beta2): -0.028668
  τ  (tau):   2.071638


### Step 6: Visualization

With our optimal parameters, we can now generate and plot the smooth yield curve. To validate our results, we will plot this curve alongside the individual market-implied yields for each bond.

To do this, we first need to calculate the **Yield-to-Maturity (YTM)** for each bond. The YTM is the single discount rate that, when applied to all of a bond's cash flows, makes their total present value equal to the bond's market price. We use a numerical root-finder (`scipy.optimize.brentq`) to solve for this rate for each bond.

The final plot will show:
- **A smooth line:** The Nelson-Siegel yield curve generated from our optimal parameters.
- **Scatter points:** The YTM of each individual bond, plotted against its maturity. 

If the scatter points lie close to the smooth line, our model is a good fit for the market data.

In [7]:
# --- Calculate YTM for each bond for the scatter plot ---
def calculate_ytm(bond):
    """Calculates the continuously compounded YTM for a single bond."""
    try:
        # Define an error function for the root-finder
        def price_error(ytm):
            pv = sum([cf * np.exp(-ytm * t) for t, cf in bond['Cashflows']])
            return pv - bond['Market_Price']
        
        # Use Brent's method to find the root (the YTM) in a reasonable range
        return brentq(price_error, -0.05, 0.25)
    except ValueError:
        # If the root-finder fails, return NaN (Not a Number)
        return np.nan

# Apply the YTM calculation to each bond
df['YTM'] = df.apply(calculate_ytm, axis=1)

# --- Prepare data for the smooth curve plot ---
max_maturity = df['Maturity_in_Years'].max()
plot_maturities = np.linspace(0, max_maturity + 1, 200) # x-axis for smooth curve
plot_yields = [nelson_siegel_rate(t, *optimal_params) * 100 for t in plot_maturities] # y-axis

# --- Create the interactive plot ---
fig = go.Figure()

# Add the smooth Nelson-Siegel curve
fig.add_trace(go.Scatter(
    x=plot_maturities,
    y=plot_yields,
    mode='lines',
    line=dict(width=3),
    name='Fitted Nelson-Siegel Curve'
))

# Add the scatter plot of individual bond YTMs
fig.add_trace(go.Scatter(
    x=df['Maturity_in_Years'],
    y=df['YTM'] * 100,
    mode='markers',
    name='Individual Bond YTMs',
    marker=dict(size=8, opacity=0.7),
    # Custom hover text shows details for each bond
    hovertemplate=(
        "<b>Maturity:</b> %{x:.2f} years<br>"
        "<b>YTM:</b> %{y:.3f}%<br>"
        "<b>Coupon:</b> %{customdata[0]:.3f}%<br>"
        "<b>Price:</b> %{customdata[1]:.4f}"
        "<extra></extra>"
    ),
    customdata=df[['Cpn', 'Market_Price']]
))

# Customize the plot's appearance
fig.update_layout(
    title=f"<b>USD Treasury Yield Curve ({VALUATION_DATE.strftime('%Y-%m-%d')})</b>",
    xaxis_title="Time to Maturity (Years)",
    yaxis_title="Yield (%)",
    yaxis_ticksuffix="%",
    legend=dict(yanchor="top", y=0.98, xanchor="left", x=0.8),
    font=dict(size=12)
)

fig.show()

### Conclusion

We have successfully constructed a smooth, continuous yield curve from a discrete set of bond data. 

The Nelson-Siegel model, combined with a numerical optimization routine, allowed us to find the four parameters that best describe the term structure of interest rates on the given valuation date. The final plot confirms that our fitted curve provides a reasonable approximation of the market, capturing the overall shape implied by the individual bond yields.